In [19]:
import numpy as np
import pandas as pd
from numpy import log2 as log
from sklearn import tree
import pydotplus
from IPython.display import Image
import time

In [20]:
eps = np.finfo(float).eps

In [21]:
dataset = {'Taste':['Salty','Spicy','Spicy','Spicy','Spicy','Sweet','Salty','Sweet','Spicy','Salty' ],
            'Temperature':['Hot','Hot','Hot','Cold','Hot','Cold','Cold','Hot','Cold','Hot'],
            'Texture':['Soft','Soft','Hard','Hard','Hard','Soft','Soft','Soft','Soft','Hard'],
            'Eat':['No','No','Yes','No','Yes','Yes','No','Yes','Yes','Yes']
          }

In [22]:
df = pd.DataFrame(dataset, columns=['Taste','Temperature','Texture','Eat'])
print(df)

   Taste Temperature Texture  Eat
0  Salty         Hot    Soft   No
1  Spicy         Hot    Soft   No
2  Spicy         Hot    Hard  Yes
3  Spicy        Cold    Hard   No
4  Spicy         Hot    Hard  Yes
5  Sweet        Cold    Soft  Yes
6  Salty        Cold    Soft   No
7  Sweet         Hot    Soft  Yes
8  Spicy        Cold    Soft  Yes
9  Salty         Hot    Hard  Yes


In [23]:
#q1
entropy_node = 0
values = df.Eat.unique()
for value in values:
    fraction = df.Eat.value_counts()[value]/len(df.Eat)
    entropy_node += -fraction * np.log2(fraction)
    
print('Parent Entropy is: ', entropy_node)

Parent Entropy is:  0.9709505944546686


In [24]:
def index(attribute):
    attribute = attribute
    target_variables  = df.Eat.unique()
    variables = df[attribute].unique()
    entropy_attribute = 0
    for variable in variables:
        entropy_each_feature = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute]==variable][df.Eat == target_variable])
            den = len(df[attribute][df[attribute]==variable])
            fraction = num/(den+eps)
            entropy_each_feature += fraction*log(fraction+eps)
        fraction2 = den/len(df)
        entropy_attribute += -fraction2*entropy_each_feature

    IG_attribute = entropy_node - abs(entropy_attribute)
    print("Entropy of ", attribute, "is: ", round(abs(entropy_attribute),3))
    print("IG ", "due to", attribute, "is: ", round(IG_attribute,3))
    
index('Taste')
index('Temperature')
index('Texture')

Entropy of  Taste is:  0.761
IG  due to Taste is:  0.21
Entropy of  Temperature is:  0.951
IG  due to Temperature is:  0.02
Entropy of  Texture is:  0.925
IG  due to Texture is:  0.046


In [25]:
#q2
x_columns = pd.get_dummies(df[['Taste','Temperature','Texture']])

for rand in range(1):
    dtree = tree.DecisionTreeClassifier(max_features = 3, random_state = rand, max_depth = 5, 
                                        criterion = 'entropy', splitter = 'best')
    model = dtree.fit(x_columns,df['Eat'])
    out_file = tree.export_graphviz(model,feature_names = list(x_columns.columns.values), class_names = ['No','Yes'], filled = True, rounded = True)
    graph = pydotplus.graph_from_dot_data(out_file)
    Image(graph.create_png())
    graph.write_png('tree.png')
    print(rand)
#     time.sleep(5)
    
    

0


In [26]:
#q3
# fraction=0
import pprint
def find_entropy(df):
    global fraction
    Class = df.keys()[-1]
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction += df[Class].value_counts()[value]/len(df[Class])
        entropy += -fraction*np.log2(fraction)
    return entropy
        

In [27]:
def find_entropy_attribute(df,attribute):
    Class = df.keys()[-1]
    target_variables = df[Class].unique()
    variables = df[attribute].unique()
    entropy2 = 0
    for variable in variables:
        entropy = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute]==variable][df[Class] == target_variable])
            den = len(df[attribute][df[attribute]==variable])
            fraction = num/(den+eps)
            entropy += -fraction*log(fraction+eps)
        fraction2 = den/len(df)
        entropy2 += -fraction2*entropy
    return abs(entropy2)


In [28]:
def find_winner(df):
    IG=[]
    for key in df.keys()[:-1]:
        IG.append(find_entropy(df)-find_entropy_attribute(df,key))
    return df.keys()[:-1][np.argmax(IG)]


In [29]:
def get_subtable(df, node, value):
    return df[df[node]==value].reset_index(drop=False)

In [30]:
def buildTree(df,d_tree=None):
    node = find_winner(df)
    attValue = np.unique(df[node])
#     attValue ['asd','xycdf']
    
    if d_tree is None:
        d_tree={}
        d_tree[node] = {}
    for value in attValue:
        subtable = get_subtable(df,node,value)
        clValue,counts = np.unique(subtable['Eat'],return_counts=True)
#         print(len(counts))
#         print(subtable)
        if len(counts)==1:
            d_tree[node][value] = clValue[0]
            
        else:
#             print(value)
            d_tree[node][value] = buildTree(subtable)
    return d_tree

dtree = buildTree(df)
print("\n\n The decision tree in dictionary form is ")
pprint.pprint(dtree)



 The decision tree in dictionary form is 
{'Taste': {'Salty': {'index': {0: 'No', 6: 'No', 9: 'Yes'}},
           'Spicy': {'index': {1: 'No', 2: 'Yes', 3: 'No', 4: 'Yes', 8: 'Yes'}},
           'Sweet': 'Yes'}}
